- parsed_chart에서 모든 song_id를 읽는다.
- 하나씩 가져와서 html로 저장한다.

In [1]:
import os
import requests
import time
import json
from tqdm import tqdm

## get song ids

In [2]:
path = "parsed_chart_html"
json_files = os.listdir(path)

In [3]:
song_ids = set()
for json_file in json_files:
    with open(os.path.join(path, json_file), "r", encoding = 'utf-8') as f:
        for line in tqdm(f):
            doc = json.loads(line)
            song_id = doc["title_num"]
            song_ids.add(song_id)
    
print("num song ids: ", len(song_ids))

5141it [00:00, 146840.34it/s]
5182it [00:00, 152397.16it/s]
5200it [00:00, 126806.01it/s]
5200it [00:00, 148538.69it/s]
5299it [00:00, 165557.64it/s]
5192it [00:00, 167445.78it/s]
5184it [00:00, 157061.44it/s]
5193it [00:00, 162247.07it/s]
5200it [00:00, 144409.96it/s]
5200it [00:00, 162469.41it/s]
5300it [00:00, 147174.41it/s]
5198it [00:00, 120852.95it/s]
5200it [00:00, 179283.54it/s]
5200it [00:00, 157541.65it/s]
5200it [00:00, 162450.05it/s]
5300it [00:00, 160618.86it/s]
5200it [00:00, 157508.65it/s]
5200it [00:00, 167706.37it/s]
5200it [00:00, 144353.57it/s]

num song ids:  12277


In [4]:
song_ids = sorted(song_ids)

## crawl song info

In [5]:
def requestsHtml(url):
    #
    PROXY_IP = "proxypool.daumsoft.com:7002"
    proxies = {"http": PROXY_IP, "https": PROXY_IP}
    
    user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36'
    headers = {"User-Agent": user_agent, "Host": "www.melon.com",
              "Cookie": "PCID=16847374038867242922976; PC_PCID=16847374038867242922976; POC=MP10; wcs_bt=s_f9c4bde066b:1684738153; _T_ANO=LN7+/1jYGuEk1pWeFf/8ws5/bdU0sBvkci0zB20oZjupLTvRok3l5dCviaJVCx9XAn8VrO3gzllm5G0oWjMpE5KMumb9bNDAviDSrJV7A5gp3nmb++/xZpPclNdvRFWh6VOEBUziQPGLhysNG3F3ZmFtpYuolLGVAj1qRtMG2dpQ4Tc7djgs/gDhPY0sGe77p7HCRdKzfha3wF9G5NnwWYXr0wD+9ZHxKnqHqrtTVEmEzxIbBAnnXTNl8Tm4ekHjlvmO5duev1gUgiC6KiOXYW0g9I7QldLTUnFkS9O8Yti5d+xw5Ec6DExLjyBJNNRP/N8jziBiPmn/2q36Yv3jig==",
              }
    #resp = requests.get(url, headers=headers, proxies=proxies)
    resp = requests.get(url, headers=headers) #, proxies=proxies)
    #
    return resp




In [6]:
# 저장된 html 목록을 읽는다.
song_html_path = "htmls/songs"
file_names = os.listdir(song_html_path)
alread_crawled_song_ids = set([_.replace(".html", "") for _ in file_names])

# 이미 parse된 songs를 읽는다.
try: 
    with open("song_infos.jsonl", "r", encoding='utf-8') as f:
        lines = f.readlines()
        docs = [json.loads(_) for _ in lines]
        parsed_song_ids = [_["song_id"] for _ in docs]
except:
    parsed_song_ids = []

In [7]:
print("song_id nums: {}".format(len(song_ids)))
print("already crawled: {}".format(len(alread_crawled_song_ids)))
print("already parsed: {}".format(len(parsed_song_ids)))

song_ids = [_ for _ in song_ids if _ not in alread_crawled_song_ids and _ not in parsed_song_ids]
print("song_ids to crawl: {}".format(len(song_ids)))

song_id nums: 12277
already crawled: 5140
already parsed: 14897
song_ids to crawl: 115


In [8]:
for song_id in tqdm(song_ids):
    
    url = "https://www.melon.com/song/detail.htm?songId={}".format(song_id)
    resp = requestsHtml(url)
    if "잘못된 경로로 접근" in resp.text:
        print("잘못된 경로: {}".format(song_id))
        time.sleep(2)
        continue
        
    #
    with open(os.path.join(song_html_path, "{}.html".format(song_id)), "w", encoding='UTF-8') as of:
        print(resp.text, file=of)
        
    # 2초이상 쉬어야 끊기지 않는다.
    time.sleep(2)

100%|██████████| 115/115 [04:34<00:00,  2.39s/it]
